<a href="https://colab.research.google.com/github/SofiaCR2/Python-basico-intermedio/blob/main/ch23_Trees.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Árboles
http://openbookproject.net/thinkcs/python/english3e/trees.html
- al igual que las listas enlazadas, los árboles se componen de nodos

## Árbol binario
 - es un árbol de uso común en el que cada nodo contiene una referencia a otros dos nodos como máximo (posiblemente Ninguno)
- estas referencias se denominan subárboles izquierdo y derecho
- como el nodo de la lista enlazada, cada nodo también contiene datos/carga
- al igual que las listas enlazadas, los árboles son estructuras de datos recursivas que se definen recursivamente:
    1. el árbol vacío, representado por Ninguno, o
    2. un nodo que contiene datos y dos referencias de árbol (subárbol izquierdo y derecho)

## Construyendo árboles
- similar a la construcción de una lista enlazada

In [1]:
class Tree:
    def __init__(self, data, left=None, right=None):
        self.cargo = data
        self.left = left
        self.right = right

    def __str__(self):
        return "{}".format(self.cargo)


### forma ascendente de construir árboles
- primero cree niños y vincúlelos al padre

In [2]:
left = Tree(2)
right = Tree(3)
tree = Tree(1, left, right)

In [3]:
tree1 = Tree(10, Tree(20), Tree(30))

In [4]:
print(tree)

1


In [5]:
print(tree1)

10


## árbol transversal
- ¡La forma natural de atravesar un árbol es recursivamente!

In [6]:
def findSum(tree):
    if not tree:
        return 0
    return tree.cargo + findSum(tree.left) + findSum(tree.right)


In [7]:
findSum(tree)

6

In [8]:
findSum(tree1)

60

## Árboles de expresión
- Los árboles son una forma natural de representar la estructura de una expresión sin ambigüedades.
- la expresión infija 1 + 2 * 3 es ambigua a menos que sepamos el orden de operación que * ocurre antes de +
- podemos usar árbol para representar la misma expresión
     - los operandos son nodos hoja
     - los nodos operadores contienen referencias a sus operandos; los operadores son binarios (dos operandos)
    
<img src="./resources/tree2.png" />

- aplicaciones:
     - traducir expresiones a sufijo, prefijo e infijo
     - los compiladores usan árboles de expresión para analizar, optimizar y traducir programas
    
- tres formas de atravesar árboles: pre-orden, en orden y post-orden

In [9]:
expression = Tree('+', Tree(1), Tree('*', Tree(2), Tree(3)))

### recorrido de árbol de pre-pedido
- los contenidos de la raíz aparecen antes que los contenidos de los hijos
- algoritmo recursivo:
    - visita el nodo
    - visita el subárbol izquierdo
    - visita el subárbol derecho

In [10]:
def preorder(tree):
    if not tree:
        return
    print(tree.cargo, end=' ')
    preorder(tree.left)
    preorder(tree.right)


In [11]:
preorder(expression)

+ 1 * 2 3 

### recorrido del árbol en orden
- el contenido del árbol aparece en orden
- algoritmo recursivo:
    - visita el subárbol izquierdo
    - visitar el nodo
    - visita el subárbol derecho

In [12]:
def inorder(tree):
    if not tree:
        return
    inorder(tree.left)
    print(tree.cargo, end=' ')
    inorder(tree.right)

In [13]:
inorder(expression)

1 + 2 * 3 

In [14]:
def inorderIndented(tree, level=0):
    if not tree:
        return
    inorderIndented(tree.right, level+1)
    print('   '*level + str(tree.cargo))
    inorderIndented(tree.left, level+1)

In [15]:
inorderIndented(expression)

      3
   *
      2
+
   1


### recorrido posterior al pedido
- algoritmo recursivo:
    1. visita el subárbol izquierdo
    2. visita el subárbol derecho
    3. visitar el nodo

In [16]:
def postorder(tree):
    if not tree:
        return
    postorder(tree.left)
    postorder(tree.right)
    print(tree.cargo, end=' ')

In [17]:
postorder(expression)

1 2 3 * + 

## construyendo un árbol de expresión
- analizar una expresión infija y construir el árbol de expresión correspondiente
- por ejemplo, (3 + 7) * 9 produce un árbol:

1. tokenizar la expresión en la lista de python? Cómo (queda como ejercicio)
 - (3 + 7) * 9 = ["(", 3, "+", 7, ")", "*", 9, "end"]
2. El token "final" es útil para evitar que el analizador lea más allá del final de la lista.

In [18]:
def get_token(token_list, expected):
    if token_list[0] == expected:
        del token_list[0]
        return True
    return False

In [19]:
# handles operands
def get_number(token_list):
    x = token_list[0]
    if not isinstance(x, int):
        return None
    del token_list[0]
    return Tree(x, None, None) # leaf node

In [20]:
token_list = [9, 11, 'end']
x = get_number(token_list)
postorder(x)

9 

In [21]:
print(token_list)

[11, 'end']


In [22]:
def get_product(token_list):
    a = get_number(token_list)
    if get_token(token_list, '*'):
        b = get_number(token_list)
        return Tree('*', a, b)
    return a

In [23]:
token_list = [9, '*', 11, 'end']
tree = get_product(token_list)

In [24]:
postorder(tree)

9 11 * 

In [25]:
token_list = [9, '+', 11, 'end']
tree = get_product(token_list)
postorder(tree)

9 

In [26]:
# adapt the function for compound product such as 3 * (5 * (7 * 9))
def get_product(token_list):
    a = get_number(token_list)
    if get_token(token_list, '*'):
        b = get_product(token_list)
        return Tree('*', a, b)
    return a

In [27]:
token_list = [2, "*", 3, "*", 5 , "*", 7, "end"]
tree = get_product(token_list)
postorder(tree)

2 3 5 7 * * * 

In [28]:
#  a sum can be a tree with + at the root, a product on the left, and a sum on the right.
# Or, a sum can be just a product.
def get_sum(token_list):
    a = get_product(token_list)
    if get_token(token_list, "+"):
        b = get_sum(token_list)
        return Tree("+", a, b)
    return a

In [ ]:
token_list = [9, "*", 11, "+", 5, "*", 7, "end"]
tree = get_sum(token_list)

In [29]:
postorder(tree)

2 3 5 7 * * * 

In [30]:
# handle parenthesis
def get_number(token_list):
    if get_token(token_list, "("):
        x = get_sum(token_list)         # Get the subexpression
        get_token(token_list, ")")      # Remove the closing parenthesis
        return x
    else:
        x = token_list[0]
        if not isinstance(x, int):
            return None
        del token_list[0]
        return Tree(x, None, None)

In [31]:
# 9 * (11 + 5) * 7
token_list = [9, "*", "(", 11, "+", 5, ")", "*", 7, "end"]
tree = get_sum(token_list)
postorder(tree)

9 11 5 + 7 * * 

## manejo de errores en expresiones malformadas

In [32]:
# handle parenthesis
def get_number(token_list):
    if get_token(token_list, "("):
        x = get_sum(token_list)         # Get the subexpression
        if not get_token(token_list, ")"): # Remove the closing parenthesis
            raise ValueError('Missing close parenthesis!')
        return x
    else:
        x = token_list[0]
        if not isinstance(x, int):
            return None
        del token_list[0]
        return Tree(x, None, None)

In [33]:
token_list = [9, "*", "(", 11, "+", 5, ")", "*", 7, "end"]
tree = get_sum(token_list)
postorder(tree)

9 11 5 + 7 * * 

## Ejercicios propuestos:

1. Modifique la función inorder para que ponga paréntesis alrededor de cada operador y par de operandos. ¿La salida es correcta y sin ambigüedades? ¿Son siempre necesarios los paréntesis?



2. Escriba una función que tome una cadena de expresión y devuelva una lista de tokens.

3. Encuentre otros lugares en las funciones del árbol de expresión donde pueden ocurrir errores y agregue declaraciones de elevación apropiadas. Pruebe su código con expresiones mal formadas.